In [233]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
from crop_image import *
import imutils
from imutils import contours as imcnts
%matplotlib inline

In [234]:
image = cv2.imread('omr.png')
cv2.imshow('image.jpg',image)
cv2.waitKey(0)

-1

In [235]:
gray_scale = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
cv2.imshow('gray scale image.jpg',gray_scale)
cv2.waitKey(0)

-1

In [236]:
"""gaussian = cv2.GaussianBlur(image,(3,3),0)
cv2.imshow('image',gaussian)
cv2.waitKey(0)"""

"gaussian = cv2.GaussianBlur(image,(3,3),0)\ncv2.imshow('image',gaussian)\ncv2.waitKey(0)"

In [237]:
edged = cv2.Canny(gray_scale,50,200)
cv2.imshow('edged.jpg',edged)
cv2.waitKey(0)

-1

In [238]:
contours,_= cv2.findContours(edged, 
    cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

print("Number of Contours found = " + str(len(contours)))
  
cv2.drawContours(image, contours, -1,(0,255,0), 3)
  
cv2.imshow('Contours.jpg', image)
cv2.waitKey(0)
image = cv2.imread('omr.png')

Number of Contours found = 4


In [239]:
#sorting all contours
if(len(contours)>0):
    contours=sorted(contours,key=cv2.contourArea,reverse=True)

In [240]:
cv2.drawContours(image, contours, 0,(0,255,0), 3)
  
cv2.imshow('Contours3.jpg', image)
cv2.waitKey(0)
image = cv2.imread('omr.png')

In [241]:
paper_contour=None
for cnt in contours:
    peri = cv2.arcLength(cnt,True)
    print(peri)
    # to approximate the given shape to polygon (high value leads to underestimation and low value leads to overestimation)
    ratio=0.02
    approx=cv2.approxPolyDP(cnt,ratio*peri,True)
    cv2.drawContours(image, [approx], 0,(0,0,255), 3)
    print(len(approx))
    coordinates=approx.ravel()
    for i in range(0,len(coordinates),2):
        x=coordinates[i]
        y=coordinates[i+1]
        cv2.putText(image, "vertex"+str(i//2), (x, y),cv2.FONT_HERSHEY_COMPLEX, 0.5, (255, 0, 0)) 
    if(len(approx)==4):
        paper_contour=approx
        break
cv2.imshow('Contours2.jpg', image)
cv2.waitKey(0)
image = cv2.imread('omr.png')

1730.0
4


In [242]:
print(paper_contour.shape)

(4, 1, 2)


In [243]:
paper= four_point_transform(image, paper_contour.reshape(4, 2))  
cv2.imshow('paper', paper)
cv2.waitKey(0)

-1

In [244]:
gray_scale_paper= four_point_transform(gray_scale, paper_contour.reshape(4, 2))  
cv2.imshow('paper', gray_scale_paper)
cv2.waitKey(0)

-1

In [245]:
_,thresholded=cv2.threshold(gray_scale_paper,0,255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)
cv2.imshow('paper1', thresholded)
cv2.waitKey(0)

-1

In [246]:
# _,thresholded=cv2.threshold(gray_scale_paper,150,255, cv2.THRESH_BINARY_INV)
# cv2.imshow('paper2', thresholded)
# cv2.waitKey(0)

In [247]:
pap_cnts,_=cv2.findContours(thresholded, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

In [248]:
def test_cnts(cnts):
    cv2.drawContours(thresholded, cnts, -1,255, 3)
  
    cv2.imshow('Contours.jpg', thresholded)
    cv2.waitKey(0)

In [249]:
pre_question_cnts=[]
value=20
for cnt in pap_cnts:
    (x,y,w,h)=cv2.boundingRect(cnt)
    aspect_ratio=w/h
    if(w>=value and h>=value and aspect_ratio>=0.9 and aspect_ratio<=1.1):
        pre_question_cnts.append(cnt)

In [250]:
print(len(pre_question_cnts),len(pap_cnts))

68 93


In [251]:
question_cnts,_=imcnts.sort_contours(pre_question_cnts,method='top-to-bottom')

In [252]:
Answer_key= {0:1, 1:4, 2:0, 3:2}
correct=0

In [253]:
print(len(question_cnts))

68


In [254]:
#test_cnts(question_cnts)

In [255]:
_,thresholded=cv2.threshold(gray_scale_paper,150,255, cv2.THRESH_BINARY_INV)
pap_cnts,_=cv2.findContours(thresholded, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

In [256]:
colors=[(255,0,0)
,(255,255,255)
,(255,0,0)
,(0,255,0)
,(255,255,0)
,(0,0,255)
,(0,255,255)
,(255,0,255)
,(192,192,192)
,(128,128,128)
,(128,0,0)
,(128,128,0)
,(0,128,0)
,(128,0,128)
,(0,128,128)
,(0,0,128),
       (255,215,0)]

In [257]:
counter=0
for (qs,i) in enumerate(np.arange(0,len(question_cnts),4)):
    curr_ques_cnts,_=imcnts.sort_contours(question_cnts[i:i+4])
    color1=colors[qs]
    cv2.drawContours(paper,curr_ques_cnts,-1,color1, 1)
    bubbled=None
    for (j,c) in enumerate(curr_ques_cnts):
        mask = np.zeros(thresholded.shape, dtype="uint8")
        #negative one at the last arguement to fill the contours
        cv2.drawContours(mask, [c], -1, 255, -1)
#         if(j==1 and qs==0):
#             cv2.imshow('mask1.jpg', mask)
#             cv2.waitKey(0)
        mask= cv2.bitwise_and(thresholded, mask)
#         if(j==1 and qs==0):
#             cv2.imshow('mask2.jpg', mask)
#             cv2.waitKey(0)
        total= cv2.countNonZero(mask)

        if bubbled is None or total >bubbled[0]:
            bubbled= (total, j)
        color= (0, 0, 255)
#     k= Answer_key[qs]    
    print(f'qestion#{qs+1}: {bubbled[1]}')
    counter+=1
#     if k == bubbled[1]:
#         correct= correct+1
print(counter)
counter=0
cv2.imshow('paper', paper)
cv2.waitKey(0)
paper= four_point_transform(image, paper_contour.reshape(4, 2)) 

qestion#1: 0
qestion#2: 0
qestion#3: 2
qestion#4: 2
qestion#5: 3
qestion#6: 0
qestion#7: 2
qestion#8: 3
qestion#9: 1
qestion#10: 3
qestion#11: 2
qestion#12: 2
qestion#13: 1
qestion#14: 2
qestion#15: 2
qestion#16: 2
qestion#17: 3
17


In [258]:
print(correct)
correct=0

0


In [259]:
cv2.imshow('Contours.jpg', thresholded)
cv2.waitKey(0)

-1

In [260]:
for i,cnt in enumerate(question_cnts):
    color1=(255,0,0) if i%2==0 else (0,0,255)
    cv2.drawContours(paper,[cnt],-1,color1, 1)
cv2.imshow('paper', paper)
cv2.waitKey(0)
paper= four_point_transform(image, paper_contour.reshape(4, 2)) 